In [1]:
import pandas as pd
import numpy as np
baseline = pd.read_csv('baseline.csv')

In [28]:
len(baseline['TMA_Acct'].unique())

9163

In [8]:
#stage_3a = list(baseline[baseline['stage'] == '3a']['TMA_Acct'])
stage_3a = list(baseline['TMA_Acct'])

In [25]:
All_egfr = pd.read_csv('All_egfr.csv')
len(All_egfr['TMA_Acct'].unique())

8186

In [26]:
egfr_3a = All_egfr[All_egfr['TMA_Acct'].isin(stage_3a)]
len(egfr_3a['TMA_Acct'].unique())

6328

In [12]:
All_stage = egfr_3a[['TMA_Acct', 'date', 'stage']]
All_stage = All_stage.drop_duplicates()
early = pd.DataFrame(All_stage.groupby(['TMA_Acct']).agg({'date': [np.min]})).reset_index()
early.columns=['TMA_Acct', 'min']
print(len(early))
early.head(2)

5647


,TMA_Acct,min
0,37,2010-01-13
1,86,2010-10-09


In [13]:
egfr_3a = pd.merge(egfr_3a, early, how = 'left', left_on = ['TMA_Acct'], right_on = ['TMA_Acct'])
egfr_3a['date'] = pd.to_datetime(egfr_3a['date'])
egfr_3a['min'] = pd.to_datetime(egfr_3a['min'])
egfr_3a.head(2)

,Unnamed: 0,TMA_Acct,date,egfr,stage,test_name,min
0,45,37,2012-05-29,17.868074,4,NaN,2010-01-13
1,46,37,2012-07-26,15.204267,4,NaN,2010-01-13


In [14]:
egfr_3a['period'] = ((egfr_3a['date'] - egfr_3a['min']).dt.days / 180).astype('int')
egfr_3a.head(2)

,Unnamed: 0,TMA_Acct,date,egfr,stage,test_name,min,period
0,45,37,2012-05-29,17.868074,4,NaN,2010-01-13,4
1,46,37,2012-07-26,15.204267,4,NaN,2010-01-13,5


In [15]:
egfr_traj = egfr_3a[['TMA_Acct', 'egfr', 'period']]

In [18]:
egfr_traj = pd.DataFrame(egfr_traj.groupby(['TMA_Acct', 'period']).agg({'egfr': np.mean})).reset_index()
egfr_traj.head(2)

,TMA_Acct,period,egfr
0,37,0,20.085950
1,37,2,20.161819


In [19]:
final = egfr_traj.pivot(index='TMA_Acct', columns='period', values='egfr')

In [20]:
final

period,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
TMA_Acct,,,,,,,,,,,,,,,,,,,,,
37,20.085950,NaN,20.161819,18.391985,17.824758,14.465363,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,22.319269,16.572148,18.044402,11.185802,17.777586,NaN,NaN,NaN,NaN,NaN,...,NaN,15.549982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,74.032583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,18.671119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,35.549025,40.582345,31.730784,31.140762,33.101918,35.761583,31.484437,35.775364,NaN,33.415709,...,NaN,32.800150,32.570549,NaN,30.612086,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50935,104.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50937,87.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50955,91.000000,NaN,89.000000,NaN,91.000000,NaN,82.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
final = final.dropna(thresh=3)

In [22]:
final

period,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
TMA_Acct,,,,,,,,,,,,,,,,,,,,,
37,20.085950,NaN,20.161819,18.391985,17.824758,14.465363,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,22.319269,16.572148,18.044402,11.185802,17.777586,NaN,NaN,NaN,NaN,NaN,...,NaN,15.549982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,35.549025,40.582345,31.730784,31.140762,33.101918,35.761583,31.484437,35.775364,NaN,33.415709,...,NaN,32.800150,32.570549,NaN,30.612086,NaN,NaN,NaN,NaN,NaN
214,43.042065,36.738713,32.116158,34.285637,28.256789,28.058991,29.763905,NaN,25.023768,26.154394,...,22.252226,NaN,21.073147,9.624839,NaN,NaN,NaN,NaN,NaN,NaN
256,59.207976,NaN,47.788777,NaN,41.904268,55.768303,55.377925,47.122076,54.990279,NaN,...,NaN,NaN,48.577804,39.735438,43.789444,41.470104,29.411691,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50841,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.000000,...,NaN,NaN,55.454247,NaN,NaN,44.172699,44.172699,NaN,NaN,NaN
50855,45.993478,39.969890,39.038786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50929,87.333333,NaN,NaN,48.250000,NaN,62.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
final.to_csv('final.csv')

In [27]:
final['TMA_Acct']

KeyError: 'TMA_Acct'